#  When is it most profitable to release movies?

## Import all of the relevant data

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Assigning the movie gross table to variable
df_movie_gross = pd.read_csv('dataframe_id_bom_movie_gross_gz.csv')
# importing budget csv 
df_budget = pd.read_csv('dataframe_id_tn_movie_budgets_gz.csv')

## Cleaning

In [3]:
df_movie_gross.head() #checking the layout and stats


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


###  Movie_Gross

In [4]:
# testing dropping 'studio' column due to irrelevance to any of 
test_drop = df_movie_gross.drop(axis=1, columns='studio', inplace=False)
#checking colunns
test_drop.columns

Index(['title', 'domestic_gross', 'foreign_gross', 'year'], dtype='object')

In [5]:
# dropping 'studio'
df_movie_gross.drop(axis=1, columns='studio', inplace=True)

In [6]:
#finding NaNs
print(df_movie_gross.isna())
print(df_movie_gross.isna().sum())

      title  domestic_gross  foreign_gross   year
0     False           False          False  False
1     False           False          False  False
2     False           False          False  False
3     False           False          False  False
4     False           False          False  False
...     ...             ...            ...    ...
3382  False           False           True  False
3383  False           False           True  False
3384  False           False           True  False
3385  False           False           True  False
3386  False           False           True  False

[3387 rows x 4 columns]
title                0
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64


In [7]:
# missing 5 values for sutdio [dropping column anyway] 
#domestic_gross is missing 28 values might just drop those rows
#foreign_gross missing 1350 values; calculating the percentage of data missing
(df_movie_gross['foreign_gross'].isna()
                 .sum()/len(df_movie_gross['foreign_gross'])) * 100

39.85828166519043

In [8]:
# 39.85% of our foreign gross values is missing, replacing with median values
# checking for why NaN's present (did these movies not get foreign/domestic 
# releases?)
is_NaN = df_movie_gross.isnull()  #This method allows me to see the rows that
                                    # have NaN values so I can compare columns
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df_movie_gross[row_has_NaN]

print(rows_with_NaN)

                                       title  domestic_gross foreign_gross  \
222                                  Flipped       1800000.0           NaN   
230               It's a Wonderful Afterlife             NaN       1300000   
254   The Polar Express (IMAX re-issue 2010)        673000.0           NaN   
267                           Tiny Furniture        392000.0           NaN   
269            Grease (Sing-a-Long re-issue)        366000.0           NaN   
...                                      ...             ...           ...   
3382                               The Quake          6200.0           NaN   
3383             Edward II (2018 re-release)          4800.0           NaN   
3384                                El Pacto          2500.0           NaN   
3385                                The Swan          2400.0           NaN   
3386                       An Actor Prepares          1700.0           NaN   

      year  
222   2010  
230   2010  
254   2010  
267   2010 

In [9]:
# Assuming that NaN's represent lack of releases in those markets, replacing 
# NaN's with 0's

df_movie_gross['domestic_gross'].fillna(0, inplace=True)
df_movie_gross['foreign_gross'].fillna(0, inplace=True)

# Checking results
print(df_movie_gross.isna().sum())

title             0
domestic_gross    0
foreign_gross     0
year              0
dtype: int64


In [10]:
# Re-checking datatypes
df_movie_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 4 columns):
title             3387 non-null object
domestic_gross    3387 non-null float64
foreign_gross     3387 non-null object
year              3387 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 106.0+ KB


In [11]:
# Need to change foreign_gross to float
# Function takes dataframe and column name as arguments. It then searches down
# every row in the given dataframe column and if it finds '$', commas, or 
# apostrophes, the function replaces them with empty space, deleting them.
# It then changes the datatype of the object from string to float.
# Finally, it returns the modified dataframe column.
def convert_amt_to_float(df, col):
    """
    Function takes dataframe and column name as arguments. 
    It then searches down
    every row in the given dataframe column and if it finds '$', commas, or 
    apostrophes, the function replaces them with empty space, deleting them.
    It then changes the datatype of the object from string to float.
    Finally, it returns the modified dataframe column.
    """
    
    df[col] = df[col].str.replace("$", "").str.replace(",", "").str.replace(
        "'","").astype('float')
    return df

df_movie_gross['foreign_gross'] = convert_amt_to_float(df_movie_gross, 
                                                     'foreign_gross')


In [12]:
# Checking
df_movie_gross['foreign_gross']

0                                       Toy Story 3
1                        Alice in Wonderland (2010)
2       Harry Potter and the Deathly Hallows Part 1
3                                         Inception
4                               Shrek Forever After
                           ...                     
3382                                      The Quake
3383                    Edward II (2018 re-release)
3384                                       El Pacto
3385                                       The Swan
3386                              An Actor Prepares
Name: foreign_gross, Length: 3387, dtype: object

In [13]:
# Still NaNs in foreign_gross, turning into 0's
df_movie_gross['foreign_gross'].fillna(0, inplace=True)

df_movie_gross['foreign_gross']

0                                       Toy Story 3
1                        Alice in Wonderland (2010)
2       Harry Potter and the Deathly Hallows Part 1
3                                         Inception
4                               Shrek Forever After
                           ...                     
3382                                      The Quake
3383                    Edward II (2018 re-release)
3384                                       El Pacto
3385                                       The Swan
3386                              An Actor Prepares
Name: foreign_gross, Length: 3387, dtype: object

In [14]:
# Checking dataframe
df_movie_gross.head(1)

,title,domestic_gross,foreign_gross,year
0,Toy Story 3,415000000.0,Toy Story 3,2010


In [15]:
# Checking for duplicates
print(df_movie_gross.duplicated())
df_movie_gross.shape

0       False
1       False
2       False
3       False
4       False
        ...  
3382    False
3383    False
3384    False
3385    False
3386    False
Length: 3387, dtype: bool


(3387, 4)

In [16]:
# No duplicates!
# Exporting to csv for teammates to use
df_movie_gross.to_csv('bom_movie_gross_clean.csv')

### Cleaning Budgets
Now that movie_gross is clean, it's time to move onto budgets

In [17]:
# Checking layout
df_budget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [18]:
# Checking datatypes
df_budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
id                   5782 non-null int64
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null object
domestic_gross       5782 non-null object
worldwide_gross      5782 non-null object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [19]:
# Need to drop 'id' and change all numericaL columns to floats or ints
df_budget.drop(axis=1, columns=['id'], inplace=True)
df_budget.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 5 columns):
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null object
domestic_gross       5782 non-null object
worldwide_gross      5782 non-null object
dtypes: object(5)
memory usage: 226.0+ KB


In [20]:
#checking for missing values
df_budget.isna().sum()

release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

In [21]:
# Need to change numeric values to floats
# Function takes dataframe and column name as arguments. It then searches down
# Calling function we used earlier
# Try with one
convert_amt_to_float(df_budget, 'production_budget')

,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,"Dec 18, 2009",Avatar,425000000.0,"$760,507,625","$2,776,345,279"
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,"$241,063,875","$1,045,663,875"
2,"Jun 7, 2019",Dark Phoenix,350000000.0,"$42,762,350","$149,762,350"
3,"May 1, 2015",Avengers: Age of Ultron,330600000.0,"$459,005,868","$1,403,013,963"
4,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,"$620,181,382","$1,316,721,747"
...,...,...,...,...,...
5777,"Dec 31, 2018",Red 11,7000.0,$0,$0
5778,"Apr 2, 1999",Following,6000.0,"$48,482","$240,495"
5779,"Jul 13, 2005",Return to the Land of Wonders,5000.0,"$1,338","$1,338"
5780,"Sep 29, 2015",A Plague So Pleasant,1400.0,$0,$0


In [22]:
# Making list of remaining columns to iterate function through

num_cols = ['domestic_gross', 'worldwide_gross']

# For loop will iterate our function only through the columns of our dataframe
# that are specified in our list.

for col in num_cols:
    df_budget = convert_amt_to_float(df_budget, col)

In [23]:
# Checking
print(df_budget.info())
df_budget.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 5 columns):
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null float64
domestic_gross       5782 non-null float64
worldwide_gross      5782 non-null float64
dtypes: float64(3), object(2)
memory usage: 226.0+ KB
None


,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09
2,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08
3,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09
4,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09


In [24]:
# Checking
print(df_budget.info())
df_budget.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 5 columns):
release_date         5782 non-null object
movie                5782 non-null object
production_budget    5782 non-null float64
domestic_gross       5782 non-null float64
worldwide_gross      5782 non-null float64
dtypes: float64(3), object(2)
memory usage: 226.0+ KB
None


,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09
1,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09
2,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08
3,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09
4,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09


### Preparing for Merge
Since we plan on merging these tables, we're going to change the names of columns and change release_date into separate columns

In [25]:
# Changing name of 'movie' to 'title' so it matches df_movie_gross
df_budget = df_budget.rename(columns={'movie' : 'title'})
# checking
df_budget.columns

Index(['release_date', 'title', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object')

In [26]:
# Using .split to create a new df with 3 columns of containing 
#each part of the string that's separated by '', respectively.  
release_split = df_budget.release_date.str.split(" ",expand=True)
release_split

,0,1,2
0,Dec,"18,",2009
1,May,"20,",2011
2,Jun,"7,",2019
3,May,"1,",2015
4,Dec,"15,",2017
...,...,...,...
5777,Dec,"31,",2018
5778,Apr,"2,",1999
5779,Jul,"13,",2005
5780,Sep,"29,",2015


In [27]:
# Merging both tables into a new one
budgets = pd.concat([release_split, df_budget], axis=1)
budgets.head()

,0,1,2,release_date,title,production_budget,domestic_gross,worldwide_gross
0,Dec,"18,",2009,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09
1,May,"20,",2011,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09
2,Jun,"7,",2019,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08
3,May,"1,",2015,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09
4,Dec,"15,",2017,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09


In [28]:
# Renaming columns
budgets = budgets.rename(columns={ 0 : 'month', 1: 'day', 2: 'year'})
budgets.head()

,month,day,year,release_date,title,production_budget,domestic_gross,worldwide_gross
0,Dec,"18,",2009,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09
1,May,"20,",2011,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09
2,Jun,"7,",2019,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08
3,May,"1,",2015,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09
4,Dec,"15,",2017,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09


In [29]:
# Changing month names to numbers to make plotting easier
monthdict = { 'Jan' : 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, "May": 5, "Jun": 6,
         'Jul' : 7, 'Aug' : 8, 'Sep' : 9, 'Oct' : 10, 'Nov' : 11, 'Dec' : 12}
                           
budgets = budgets.replace({'month': monthdict})
budgets.head()

,month,day,year,release_date,title,production_budget,domestic_gross,worldwide_gross
0,12,"18,",2009,"Dec 18, 2009",Avatar,425000000.0,760507625.0,2.776345e+09
1,5,"20,",2011,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.0,241063875.0,1.045664e+09
2,6,"7,",2019,"Jun 7, 2019",Dark Phoenix,350000000.0,42762350.0,1.497624e+08
3,5,"1,",2015,"May 1, 2015",Avengers: Age of Ultron,330600000.0,459005868.0,1.403014e+09
4,12,"15,",2017,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.0,620181382.0,1.316722e+09


In [30]:
# Don't need 'day' and 'release_date' column
budgets = budgets.drop(['day', 'release_date'], axis=1)
budgets.columns

Index(['month', 'year', 'title', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object')

In [31]:
# Checking for duplicates
print(budgets.shape)
budgets.duplicated().any()

(5782, 6)


False

In [32]:
# No duplicates!
# Exporting table to csv for teamates to use
budgets.to_csv('tn_movie_budgets_clean.csv')

### Merging the Tables

In [33]:
# Merging budgets with movie_gross
# We want our merge the tables so that we add the titles together, thus
# increasing our sample size.  We also only want to merge what they have in
# common
profit = budgets.set_index('title').join(df_movie_gross.set_index('title'),
                                         rsuffix='_gr',
                                                how='inner')
profit.head()

,month,year,production_budget,domestic_gross,worldwide_gross,domestic_gross_gr,foreign_gross,year_gr
title,,,,,,,,
10 Cloverfield Lane,3,2016,5000000.0,72082999.0,108286422.0,72100000.0,10 Cloverfield Lane,2016
12 Strong,1,2018,35000000.0,45819713.0,71118378.0,45800000.0,12 Strong,2018
12 Years a Slave,10,2013,20000000.0,56671993.0,181025343.0,56700000.0,12 Years a Slave,2013
127 Hours,11,2010,18000000.0,18335230.0,60217171.0,18300000.0,127 Hours,2010
13 Hours: The Secret Soldiers of Benghazi,1,2016,50000000.0,52853219.0,69411370.0,52900000.0,13 Hours: The Secret Soldiers of Benghazi,2016


In [34]:
# We already have a 'year' column and there is conflicting data in our 
# respective domestic_gross columns.  We should also reset the index.
profit = profit.reset_index()

In [35]:
profit = profit.drop('year_gr', axis=1)
profit.columns

Index(['title', 'month', 'year', 'production_budget', 'domestic_gross',
       'worldwide_gross', 'domestic_gross_gr', 'foreign_gross'],
      dtype='object')

## Filtering Relevant Data

In [36]:
# set table to variable
df = pd.read_csv('joined_gross_budget_cleaned.csv')

In [37]:
df.head() #checking the layout

,title,month,year,production_budget,domestic_gross,worldwide_gross,net_prof,net_margin_%
0,(500) Days of Summer,7.0,2009,7500000.0,32425665.0,34439060.0,26939060.0,78.22
1,10 Cloverfield Lane,3.0,2016,5000000.0,72082999.0,108286422.0,103286422.0,95.38
2,10 Things I Hate About You,3.0,1999,13000000.0,38177966.0,60413950.0,47413950.0,78.48
3,"10,000 B.C.",3.0,2008,105000000.0,94784201.0,269065678.0,164065678.0,60.98
4,12 Monkeys,12.0,1995,29000000.0,57141459.0,168841459.0,139841459.0,82.82


In [38]:
# checking for datatypes in table
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3657 entries, 0 to 3656
Data columns (total 8 columns):
title                3657 non-null object
month                3657 non-null float64
year                 3657 non-null int64
production_budget    3657 non-null float64
domestic_gross       3657 non-null float64
worldwide_gross      3657 non-null float64
net_prof             3657 non-null float64
net_margin_%         3657 non-null float64
dtypes: float64(6), int64(1), object(1)
memory usage: 228.7+ KB


## Tidying up
Everything looks great, but the column 'month' is a float.  While that isn't too much of a problem, I intend to use this column as my x-axis in my plots.  From a style point of view, I don't want decimals there, so I will be changing that to an integer format.